In [60]:
from sklearn.linear_model import LinearRegression
import numpy as np
from typing import Callable
import torch
import torch.nn as nn
import torch.nn.functional as F

In [100]:
# pyTorch neural network class

class myPytorchNetwork(nn.Module):
    def __init__(self, layers):
        super(myPytorchNetwork, self).__init__()
        self.fc_layers = nn.ModuleList([nn.Linear(layers[i], layers[i + 1]) for i in range(len(layers) - 1)])

    def forward(self, x):
        x = torch.flatten(x, 1)

        # TODO: I'm using relus everywhere for now, we need to change it to be adjustable
        for layer in self.fc_layers[:-1]:
            x = F.relu(layer(x))

        # No need for activation in the last layer
        return self.fc_layers[-1](x)


def train_torch_network(network: myPytorchNetwork, dataset, max_epochs, batch_size, train_ratio=0.7,
                        learning_rate=0.001, momentum=0.9, silent=False):
    loss_fn = torch.nn.MSELoss()
    # TODO: think of making the learning rate adaptive here, e.g. by using pytorch LR scheduler
    optimizer = torch.optim.SGD(network.parameters(), lr=learning_rate, momentum=momentum)
    split_index = int(dataset.shape[0] * train_ratio)
    train, validation = dataset[:split_index, :], torch.from_numpy(dataset[split_index:, :])

    for epoch in range(max_epochs):
        np.random.shuffle(train)
        torch_train = torch.from_numpy(train)
        batch_start_idx = 0

        batches_loss = []
        # This way we skip last samples if there are less than batch_size of them
        while batch_start_idx + batch_size <= torch_train.shape[0]:
            optimizer.zero_grad()
            outputs = network(torch_train[batch_start_idx:batch_start_idx + batch_size, :-1])
            loss = loss_fn(torch.flatten(outputs), torch_train[batch_start_idx:batch_start_idx + batch_size, -1])
            batches_loss.append(loss.item())

            loss.backward()

            optimizer.step()
            batch_start_idx += batch_size

        # Now, check the loss on validation dataset
        validation_output = torch.flatten(network(validation[:, :-1]))
        validation_loss = loss_fn(validation_output, validation[:, -1])
        if not silent:
            print(
                f"Epoch: {epoch}. \nLoss on training: {np.mean(batches_loss)} \nLoss on validation: {validation_loss} \n##########")


In [56]:
def run_linear_regression(train_inputs, train_outputs, test_inputs) -> np.array:
    """
    Train a linear regression on train_inputs, and predict data for test_inputs
    """
    reg = LinearRegression().fit(train_inputs, train_outputs)
    prediction = reg.predict(test_inputs)
    return prediction


def run_pytorch_network(network, train_inputs, train_outputs, test_inputs) -> np.array:
    """
    Train a pytorch network and predict outputs for test_inputs
    """
    train_torch_network(network)


def measure_model_error_multiple_times(model: Callable, dataset: np.array, train_ratio=0.85, num_runs=5):
    """
    :param model: a callable that accepts train inputs, train outputs, test inputs, and produces the prediction for test inputs
    :param dataset: dataset to train on of shape (n_samples, n_features), where the last column is the value to be predicted
    :param train_ratio: how much data to put into the training dataset
    :param num_runs: number of runs with reshuffled dataset
    :return: mean MSE through all the runs
    """
    split_index = int(dataset.shape[0] * train_ratio)
    error_sum = 0
    for _ in range(num_runs):
        np.random.shuffle(dataset)
        train = dataset[:split_index, :]
        test = dataset[split_index:, :]

        prediction = model(train[:, :-1], train[:, -1], test[:, :-1])
        mse = np.mean((prediction - test[:, -1]) ** 2)
        print(f"Model MSE on test: {mse}")
        error_sum += mse
    return error_sum / num_runs

In [88]:
# Load the data. TODO: add all the datasets here
turbine = np.genfromtxt("processed_datasets/turbine.csv", dtype=np.float32, delimiter=',', skip_header=1)
turbine_input, turbine_output = turbine[:, :4], turbine[:, 4]

In [64]:
# Trying a couple of times with different splits into training and test data for a better understanging
measure_model_error_multiple_times(run_linear_regression, turbine)

Model MSE: 0.0012760517420247197
Model MSE: 0.0017902901163324714
Model MSE: 0.0015872992807999253
Model MSE: 0.0017255778657272458
Model MSE: 0.0016643449198454618


0.0016087127849459648

In [89]:
turbine.shape

(451, 5)

In [99]:
torch_nn = myPytorchNetwork([4, 8, 10, 1])
print(torch_nn)
train_torch_network(torch_nn, turbine, 20, 10, learning_rate=0.1)

myPytorchNetwork(
  (fc_layers): ModuleList(
    (0): Linear(in_features=4, out_features=8, bias=True)
    (1): Linear(in_features=8, out_features=10, bias=True)
    (2): Linear(in_features=10, out_features=1, bias=True)
  )
)
Epoch: 0. 
Loss on training: 0.09286606654284461 
Loss on validation: 0.041980527341365814 
##########
Epoch: 1. 
Loss on training: 0.01773180087818013 
Loss on validation: 0.003901812946423888 
##########
Epoch: 2. 
Loss on training: 0.005159017496607116 
Loss on validation: 0.00335987051948905 
##########
Epoch: 3. 
Loss on training: 0.0034607121037439474 
Loss on validation: 0.005712843965739012 
##########
Epoch: 4. 
Loss on training: 0.003237178748203141 
Loss on validation: 0.002567701507359743 
##########
Epoch: 5. 
Loss on training: 0.0023461145089728936 
Loss on validation: 0.003956796135753393 
##########
Epoch: 6. 
Loss on training: 0.0027781967636978915 
Loss on validation: 0.003011946566402912 
##########
Epoch: 7. 
Loss on training: 0.00227568002470